In [1]:
from tqdm.auto import tqdm 
import pandas as pd
import numpy as np
import xarray as xr
import glob
%matplotlib inline

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [183]:
datapath="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/"
output="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/"
target="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/outputs/targets/"

In [210]:
tracklist = sorted(glob.glob(path+'tracks_nio/*_2001*'))

In [211]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20010521.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20011008.csv']

In [216]:
dm1=xr.open_mfdataset([datapath+'/vor/vor_2001.nc',datapath+'/rhum/rhum_2001.nc'])

In [217]:
#ssingle level variables
def output_indices(TCtrack=None,ERA5date=None,ERA5hour=None):
    allindices = []
    for timeidx in range(len(TCtrack)):#len(track['time'])):
        datetrack,hourtrack = TCtrack['time'][timeidx].split(':')[0],TCtrack['time'][timeidx].split(':')[1][0:2]
        ####################################################################################################
        # Find the indices in ERA5 data with the same date as track
        ####################################################################################################
        dateind = []
        for ind,obj in enumerate(ERA5date):
            if obj==datetrack:
                dateind.append(ind)
        del ind,obj
        hourind = []
        hourextract = ERA5hour[int(np.min(np.asarray(dateind))):int(np.max(np.asarray(dateind)))+1]
        for ind,obj in enumerate(hourextract):
            if obj==hourtrack:            
                hourind.append(ind)
        allindices.append((int(np.min(np.asarray(dateind))),int(hourind[0])))
    return allindices

def extract_var(dataset=None,var='var138',indices=None):
    extractedvar = []
    for i in (range(len(indices))):
        realindex = indices[i][0]+indices[i][1]
        extractedvar.append(dataset[var][int(realindex),...].data)
    return np.asarray(extractedvar)

def largearea(dataset=None,invar=None,indices=None):
    if len(invar.shape) != 3:
        invar = np.squeeze(invar)
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices)))),
    attrs=dict(description="coords with matrices"),)
    
    var_out=np.zeros((len(indices),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        try:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
        except:
            var_out[it,:,:]=ds['variable'][it,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out

def largearea_withpres(dataset=None,invar=None,indices=None):
    ds = xr.Dataset(
    data_vars=dict(variable=(["time","plev","lat","lon"], invar)),#mysvar[0])),
    coords=dict(lat=(["lat"], dataset.lat.data),lon=(["lon"], dataset.lon.data),time=(["time"], np.linspace(0,len(indices)-1,len(indices))),
               plev=(["plev"],dataset.plev.data)),
    attrs=dict(description="coords with matrices"),)
    var_out=np.zeros((len(indices),len(dm1.plev.data),64,64))
    for it in range(len(indices)):
        latn, lats, lone, lonw = tc_orad[it,:]
        for ip in range(len(dm1.plev.data)):
            try:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))
            except:
                var_out[it,ip,:,:]=ds['variable'][it,ip,:,:].sel(lat=slice(lats,latn),lon=slice(lone,lonw))[0:64,0:64]
    return var_out 

In [218]:
###
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/svars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2001)

TCs_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    svarname1 = ['outu10','outv10','out2mdewtmp','out2mtmp','outconv_ppt','outtot_cld_ice',\
           'outtot_cldwtr','outtot_cld_rain','outvi_div_cld_froz_wtr','outvi_div_cld_liq_wtr','outvi_div_gpot_flux',\
           'outvi_div_ke_flux','outvi_div_mass_flux','outvi_div_moisture_flux','outvi_div_olr_flux','outvi_div_tot_enrgy_flux',\
           'outvi_ke','outvi_pe_inte','outvi_pe_ie_latentenrgy','outvi_temp','outvi_olr','outvi_tot_enrgy','outvi_moisture_div']
   
    svarname = ['u10','v10','2mdewtmp','2mtmp','conv_ppt','tot_cld_ice',\
           'tot_cldwtr','tot_cld_rain','vi_div_cld_froz_wtr','vi_div_cld_liq_wtr','vi_div_gpot_flux',\
           'vi_div_ke_flux','vi_div_mass_flux','vi_div_moisture_flux','vi_div_olr_flux','vi_div_tot_enrgy_flux',\
           'vi_ke','vi_pe_inte','vi_pe_ie_latentenrgy','vi_temp','vi_olr','vi_tot_enrgy','vi_moisture_div']
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCs_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [219]:
###radiation variables
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_dataset(datapath+'/slev_vars/radvars_'+str(year)+'.nc')
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2001)

TCr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outcape','outinst_10m_wnd_gst','outinst_moisture_flux','outinst_ssh_flux','outsurfmean_swr_flux','outsurfmean_lhf',\
           'outsurfmean_lwr_flux','outsurfmean_shf','outdwnwrdmean_swr_flux','outtopmean_lwr_flux','outtopmean_swr_flux',\
           'outvimean_moisture_div','outsurf_lhf','outsurf_shf','outtot_suprcool_liqwtr','outtot_wtr_vpr']
    
    
    svarname = ['cape','inst_10m_wnd_gst','inst_moisture_flux','inst_ssh_flux','surfmean_swr_flux','surfmean_lhf',\
           'surfmean_lwr_flux','surfmean_shf','dwnwrdmean_swr_flux','topmean_lwr_flux','topmean_swr_flux',\
           'vimean_moisture_div','surf_lhf','surf_shf','tot_suprcool_liqwtr','tot_wtr_vpr']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCr_ts.append(tsdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [220]:
#########precip, mslp, sst variables############
def createmask(dm=None,irad=None,orad=None,lonselect=None,latselect=None):
    mask = []
    for ti in range(len(tc_irad)):
        lonselect = dm.lon.sel(lon=slice(orad[ti,:][2],orad[ti,:][3])).data
        latselect = np.flipud(dm.lat.sel(lat=slice(orad[ti,:][1],orad[ti,:][0])).data)
        if (lonselect.shape != 64) or (latselect.shape != 64):
            lon2d,lat2d = np.meshgrid(lonselect[0:64],latselect[0:64])
        else:
            lon2d,lat2d = np.meshgrid(lonselect,latselect)
        #############################################################################################
        latcriteria = np.logical_and(lat2d>irad[ti][0],lat2d<irad[ti][1])
        loncriteria = np.logical_and(lon2d>irad[ti][2],lon2d<irad[ti][3])
        allcriteria = np.logical_and(loncriteria,latcriteria)
        mask.append(allcriteria)
    return mask

def readyear(year=None):
    dm2 = xr.open_mfdataset([datapath+'/prate/prates_2001.nc',datapath+'/mslp/mslp_2001.nc',datapath+'/sst/sst_2001.nc'])
    tracklist = sorted(glob.glob('/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/*_'+str(year)+'*'))
    era5_date = [str(dm2.time[i].data).split('T')[0] for i in range(len(dm2.time))]
    era5_hour = [str(dm2.time[i].data).split('T')[1][0:2] for i in range(len(dm2.time))]
    return dm2,tracklist,era5_date,era5_hour

dm2,tracklist,era5_date,era5_hour = readyear(2001)

TCpr_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mysvar = [extract_var(dataset=dm2,var=obj,indices=indices_store) for obj in (list(dm2.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallsvarout = [largearea(dm2,mysvar[i],indices_store) for i in (range(len(mysvar)))]
    
    svarname1 = ['outmslp','outconv_rrate','outls_rrate','outmn_conv_prate',\
                 'outmn_ls_prate','outmn_tot_prate','outsst']
    
    svarname = ['mslp','conv_rrate','ls_rrate','mn_conv_prate','mn_ls_prate','mn_tot_prate','sst']
    
    svardict = {varnameobj:varobj for (varnameobj,varobj) in zip(svarname,smallsvarout)}
    
    lonselect = dm2.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm2.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)

    mymask = createmask(dm=dm2,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    tsdict = {}
    for ind,obj in (enumerate(svarname)):
        tslist = [svardict[svarname[ind]][i,...][~mymask[i]] for i in range(len(mymask))]
        tsdict[svarname1[ind]] = [np.nanmean(obj) for obj in tslist]
    #############################################################################################
    TCpr_ts.append(tsdict)

  0%|          | 0/2 [00:00<?, ?it/s]

In [221]:
#Pressure vars - vor hum
TCp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvort','outrhum']
    pvarname = ['vort','rhum']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [222]:
dm1=xr.open_dataset(datapath+'/gpot/gpot_2001.nc')

In [223]:
# Pressure vars - geopotential
TCgp_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outgpot']
    pvarname = ['gpot']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCgp_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [224]:
dm1=xr.open_dataset(datapath+'/vvel/w_wnd_2001.nc')

In [225]:
#Pressure vars - vertical velocity
TCvv_ts = []
for TCobj in tqdm(tracklist):
    track=pd.read_csv(TCobj,delimiter=r",")
    lon1=track['lon'].to_numpy()
    lat1=track['lat'].to_numpy()
    pos = arr = np.stack((lat1, lon1), axis=1)
    ###########################################################################
    indices_store = output_indices(track,era5_date,era5_hour)
    ###########################################################################
    mypvar = [extract_var(dataset=dm1,var=obj,indices=indices_store) for obj in (list(dm1.keys()))]
    ###########################################################################
    tc_irad=np.empty((len(indices_store),4))
    tc_irad[:,0] = pos[:,0]-2
    tc_irad[:,1] = pos[:,0]+2
    tc_irad[:,2] = pos[:,1]-2
    tc_irad[:,3] = pos[:,1]+2
    
    tc_orad=np.empty((len(indices_store),4))
    tc_orad[:,0] = pos[:,0]-8
    tc_orad[:,1] = pos[:,0]+8
    tc_orad[:,2] = pos[:,1]-8
    tc_orad[:,3] = pos[:,1]+8
    ###########################################################################
    smallpvarout = [largearea_withpres(dm1,mypvar[i],indices_store) for i in (range(len(mypvar)))]    
    pvarname1 = ['outvvel']
    pvarname = ['vvel']
    pvardict = {varnameobj:varobj for (varnameobj,varobj) in zip(pvarname,smallpvarout)}
    
    lonselect = dm1.lon.sel(lon=slice(tc_orad[0,:][2],tc_orad[0,:][3])).data
    latselect = np.flipud(dm1.lat.sel(lat=slice(tc_orad[0,:][1],tc_orad[0,:][0])).data)
    lon2d,lat2d = np.meshgrid(lonselect,latselect)
    mymask = createmask(dm=dm1,irad=tc_irad,orad=tc_orad,lonselect=lonselect,latselect=latselect)
    #############################################################################################
    ts_pdict = {}
    for ind,obj in (enumerate(pvarname)):
        pvarTS_store = []
        for plevv in range(len(dm1.plev.data)):
            tempvar = pvardict[pvarname[ind]][:,plevv,...]
            tempts = [tempvar[i,...][~mymask[i]] for i in range(len(mymask))]
            tempTSERIES = [np.nanmean(obj) for obj in tempts]
            pvarTS_store.append(tempTSERIES)
        ts_pdict[pvarname1[ind]] = np.asarray(pvarTS_store).transpose()
    #############################################################################################
    TCvv_ts.append(ts_pdict)
#myvar = [extract_var(var=obj,indices=indices_store) for obj in vars_dm1]

  0%|          | 0/2 [00:00<?, ?it/s]

In [226]:
preslv = [str(int(obj)) for obj in dm1.plev.data/100]

In [227]:
dx=xr.open_dataset(datapath+'/tlevs/tlevs_2001.nc')

In [228]:
plev=[str(int(obj)) for obj in dx.plev.data/100]

In [229]:
tracklist

['/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20010521.csv',
 '/work/FAC/FGSE/IDYST/tbeucler/default/saranya/create_ts/tracks_nio/nio_20011008.csv']

In [230]:
#storms 1
mainas12001=pd.read_csv(output+'inner/inner_nio_era5_as12001.csv',delimiter=r",")
mainas12001.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainas12001=mainas12001.drop('a', axis=1)

#pdf_as12001=pd.read_csv(target+'2011_pmin_nio_as12001.csv',delimiter=r",")
#pdf_as12001.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#pdf_as12001=pdf_as12001.drop('a', axis=1)

#extra=pd.read_csv(output+'nio/outder_2011_nio_as12001.csv',delimiter=r",")
#extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#extra=extra.drop('a', axis=1)


slev_as12001=pd.DataFrame.from_dict(TCs_ts[0])
rlev_as12001=pd.DataFrame.from_dict(TCr_ts[0])
prlev_as12001=pd.DataFrame.from_dict(TCpr_ts[0])
prlev_as12001['outmslp']=prlev_as12001['outmslp']/100

#ts1_as12001=pd.concat([mainas12001,extra,slev_as12001,rlev_as12001,prlev_as12001], axis=1, join='inner')
ts1_as12001=pd.concat([mainas12001,slev_as12001,rlev_as12001,prlev_as12001], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[0]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[0]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[0]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[0]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_as12001=pd.DataFrame.from_dict(alldict1)

#ts_as12001=pd.concat([pdf_as12001,ts1_as12001,plev_as12001], axis=1, join='inner')
ts_as12001=pd.concat([ts1_as12001,plev_as12001], axis=1, join='inner')
ts_as12001

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,2.797020,-1.300709,297.503342,300.435463,0.000992,0.457743,0.077123,0.168191,2.278131e-06,-4.065223e-07,...,-0.029705,-0.044042,-0.066115,-0.087685,-0.085561,-0.074602,-0.051789,-0.033572,-0.022276,-0.017418
1,3.219173,-0.978985,297.747986,300.189552,0.000926,0.449180,0.071876,0.155215,4.057074e-07,-7.695053e-08,...,-0.043450,-0.056483,-0.071663,-0.085794,-0.082817,-0.071525,-0.057668,-0.042821,-0.036999,-0.034332
2,2.672218,-1.660519,297.782349,300.266228,0.001222,0.416765,0.079823,0.116814,-1.804437e-08,-1.251930e-06,...,-0.071566,-0.082827,-0.076554,-0.062296,-0.041045,-0.026154,-0.020540,-0.013239,-0.008206,-0.005110
3,2.963585,-0.259609,298.031876,300.487274,0.001619,0.369974,0.098702,0.112605,2.079643e-06,-6.742775e-07,...,-0.036461,-0.034604,-0.012681,0.016203,0.045634,0.047715,0.028901,0.018179,0.013773,0.014854
4,3.044754,-0.760643,298.218849,300.192093,0.001579,0.489292,0.147114,0.198503,4.214235e-06,-6.970949e-07,...,-0.020463,-0.012012,0.007420,0.027343,0.048446,0.042108,0.001549,-0.009546,-0.008148,-0.004489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,1.808327,0.303658,298.515915,300.787300,0.000422,0.043984,0.161987,0.111859,4.921458e-07,4.868489e-07,...,-0.011654,-0.012715,-0.006971,0.010783,0.016985,0.003359,-0.013662,-0.004843,0.016866,0.026316
63,0.583748,-0.275944,298.645134,301.783989,0.000325,0.069014,0.135358,0.108026,-2.275629e-06,-1.118868e-06,...,-0.016906,-0.022113,-0.022949,0.000352,0.001698,-0.020632,-0.025549,-0.017224,-0.000044,0.007499
64,0.986526,1.515474,298.449669,302.721069,0.000246,0.122222,0.206177,0.187214,1.917942e-06,-8.493662e-07,...,-0.017400,-0.013201,-0.001739,0.009581,-0.003903,-0.036989,-0.049990,-0.049936,-0.039385,-0.032919
65,1.568237,3.170792,297.796585,304.296021,0.000241,0.107788,0.163889,0.205021,7.151044e-06,7.633353e-07,...,-0.008327,0.003808,0.024853,0.015244,-0.017630,-0.066234,-0.075738,-0.072792,-0.064121,-0.061553


In [231]:
df1=ts_as12001
#df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
#df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,vi_div_mass_flux,vi_div_moisture_flux,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,297.503342,300.435463,0.000992,0.457743,0.077123,0.168191,2.278131e-06,-4.065223e-07,0.001257,-0.000325,...,-0.029705,-0.044042,-0.066115,-0.087685,-0.085561,-0.074602,-0.051789,-0.033572,-0.022276,-0.017418
1,297.747986,300.189552,0.000926,0.449180,0.071876,0.155215,4.057074e-07,-7.695053e-08,0.002727,-0.000205,...,-0.043450,-0.056483,-0.071663,-0.085794,-0.082817,-0.071525,-0.057668,-0.042821,-0.036999,-0.034332
2,297.782349,300.266228,0.001222,0.416765,0.079823,0.116814,-1.804437e-08,-1.251930e-06,-0.000405,-0.000303,...,-0.071566,-0.082827,-0.076554,-0.062296,-0.041045,-0.026154,-0.020540,-0.013239,-0.008206,-0.005110
3,298.031876,300.487274,0.001619,0.369974,0.098702,0.112605,2.079643e-06,-6.742775e-07,-0.002027,-0.000517,...,-0.036461,-0.034604,-0.012681,0.016203,0.045634,0.047715,0.028901,0.018179,0.013773,0.014854
4,298.218849,300.192093,0.001579,0.489292,0.147114,0.198503,4.214235e-06,-6.970949e-07,0.001147,-0.000615,...,-0.020463,-0.012012,0.007420,0.027343,0.048446,0.042108,0.001549,-0.009546,-0.008148,-0.004489
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,298.515915,300.787300,0.000422,0.043984,0.161987,0.111859,4.921458e-07,4.868489e-07,-0.002508,-0.000246,...,-0.011654,-0.012715,-0.006971,0.010783,0.016985,0.003359,-0.013662,-0.004843,0.016866,0.026316
63,298.645134,301.783989,0.000325,0.069014,0.135358,0.108026,-2.275629e-06,-1.118868e-06,-0.000061,-0.000311,...,-0.016906,-0.022113,-0.022949,0.000352,0.001698,-0.020632,-0.025549,-0.017224,-0.000044,0.007499
64,298.449669,302.721069,0.000246,0.122222,0.206177,0.187214,1.917942e-06,-8.493662e-07,0.000273,-0.000595,...,-0.017400,-0.013201,-0.001739,0.009581,-0.003903,-0.036989,-0.049990,-0.049936,-0.039385,-0.032919
65,297.796585,304.296021,0.000241,0.107788,0.163889,0.205021,7.151044e-06,7.633353e-07,0.000278,-0.000256,...,-0.008327,0.003808,0.024853,0.015244,-0.017630,-0.066234,-0.075738,-0.072792,-0.064121,-0.061553


In [232]:
df1.to_csv(output+'timeseries_era5_nio_as12001.csv')

In [233]:
#storms 1
mainas22001=pd.read_csv(output+'inner/inner_nio_era5_as22001.csv',delimiter=r",")
mainas22001.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainas22001=mainas22001.drop('a', axis=1)

#pdf_as22001=pd.read_csv(target+'2011_pmin_nio_as22001.csv',delimiter=r",")
#pdf_as22001.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#pdf_as22001=pdf_as22001.drop('a', axis=1)

#extra=pd.read_csv(output+'nio/outder_2011_nio_as22001.csv',delimiter=r",")
#extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#extra=extra.drop('a', axis=1)


slev_as22001=pd.DataFrame.from_dict(TCs_ts[1])
rlev_as22001=pd.DataFrame.from_dict(TCr_ts[1])
prlev_as22001=pd.DataFrame.from_dict(TCpr_ts[1])
prlev_as22001['outmslp']=prlev_as22001['outmslp']/100

#ts1_as22001=pd.concat([mainas22001,extra,slev_as22001,rlev_as22001,prlev_as22001], axis=1, join='inner')
ts1_as22001=pd.concat([mainas22001,slev_as22001,rlev_as22001,prlev_as22001], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[1]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[1]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[1]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[1]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_as22001=pd.DataFrame.from_dict(alldict1)

#ts_as22001=pd.concat([pdf_as22001,ts1_as22001,plev_as22001], axis=1, join='inner')
ts_as22001=pd.concat([ts1_as22001,plev_as22001], axis=1, join='inner')
ts_as22001

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,2.797020,-1.300709,297.503342,300.435463,0.000992,0.457743,0.077123,0.168191,2.278131e-06,-4.065223e-07,...,0.013642,0.020046,0.006611,0.003401,-0.003535,-0.011124,-0.017474,-0.022932,-0.021963,-0.020007
1,3.219173,-0.978985,297.747986,300.189552,0.000926,0.449180,0.071876,0.155215,4.057074e-07,-7.695053e-08,...,0.006745,0.011196,0.004407,0.005323,-0.006845,-0.019398,-0.034454,-0.031329,-0.028143,-0.026366
2,2.672218,-1.660519,297.782349,300.266228,0.001222,0.416765,0.079823,0.116814,-1.804437e-08,-1.251930e-06,...,0.008412,0.022439,0.027043,0.016089,-0.005273,-0.007740,-0.014920,-0.005310,0.001874,0.002673
3,2.963585,-0.259609,298.031876,300.487274,0.001619,0.369974,0.098702,0.112605,2.079643e-06,-6.742775e-07,...,-0.006421,0.005606,0.013276,0.008117,-0.010802,-0.003311,-0.000975,0.007085,0.016742,0.020204
4,3.044754,-0.760643,298.218849,300.192093,0.001579,0.489292,0.147114,0.198503,4.214235e-06,-6.970949e-07,...,-0.010379,-0.002832,0.013186,0.025598,0.014989,0.005386,-0.020116,-0.017944,-0.009105,-0.003395
5,3.803032,-0.391453,298.172112,299.935692,0.001363,0.417446,0.253442,0.404910,5.655689e-06,-5.851034e-07,...,-0.001985,0.006779,0.021545,0.036025,0.024742,0.018708,-0.014994,-0.018865,-0.011340,-0.004589
6,2.532806,-0.018833,298.005112,300.028671,0.001378,0.317016,0.201483,0.304174,6.614486e-06,-8.600764e-07,...,-0.007821,-0.013175,0.003263,0.025168,0.022397,0.018198,-0.000035,-0.000706,0.010331,0.018019
7,2.415497,1.057812,298.150940,300.186386,0.001652,0.482024,0.207725,0.317189,1.047959e-05,-2.657296e-06,...,-0.000602,-0.009430,0.003724,0.019110,0.019226,0.015106,0.004625,0.003430,0.011957,0.018585
8,2.148190,1.878307,298.180840,300.633049,0.001718,0.612303,0.246403,0.406686,2.709043e-05,-1.333887e-06,...,-0.000828,-0.008117,-0.001176,0.005145,0.002854,-0.003733,-0.015604,-0.017871,-0.013112,-0.008933
9,3.463573,1.598438,298.302040,300.461037,0.001550,0.561260,0.337152,0.500936,2.518040e-05,5.151960e-07,...,0.001008,0.001679,0.003366,-0.006648,-0.011467,-0.023606,-0.038762,-0.037297,-0.030485,-0.026019


In [234]:
df1=ts_as22001
#df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
#df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,vi_div_mass_flux,vi_div_moisture_flux,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,297.503342,300.435463,0.000992,0.457743,0.077123,0.168191,2.278131e-06,-4.065223e-07,0.001257,-0.000325,...,0.013642,0.020046,0.006611,0.003401,-0.003535,-0.011124,-0.017474,-0.022932,-0.021963,-0.020007
1,297.747986,300.189552,0.000926,0.449180,0.071876,0.155215,4.057074e-07,-7.695053e-08,0.002727,-0.000205,...,0.006745,0.011196,0.004407,0.005323,-0.006845,-0.019398,-0.034454,-0.031329,-0.028143,-0.026366
2,297.782349,300.266228,0.001222,0.416765,0.079823,0.116814,-1.804437e-08,-1.251930e-06,-0.000405,-0.000303,...,0.008412,0.022439,0.027043,0.016089,-0.005273,-0.007740,-0.014920,-0.005310,0.001874,0.002673
3,298.031876,300.487274,0.001619,0.369974,0.098702,0.112605,2.079643e-06,-6.742775e-07,-0.002027,-0.000517,...,-0.006421,0.005606,0.013276,0.008117,-0.010802,-0.003311,-0.000975,0.007085,0.016742,0.020204
4,298.218849,300.192093,0.001579,0.489292,0.147114,0.198503,4.214235e-06,-6.970949e-07,0.001147,-0.000615,...,-0.010379,-0.002832,0.013186,0.025598,0.014989,0.005386,-0.020116,-0.017944,-0.009105,-0.003395
5,298.172112,299.935692,0.001363,0.417446,0.253442,0.404910,5.655689e-06,-5.851034e-07,0.002472,-0.000621,...,-0.001985,0.006779,0.021545,0.036025,0.024742,0.018708,-0.014994,-0.018865,-0.011340,-0.004589
6,298.005112,300.028671,0.001378,0.317016,0.201483,0.304174,6.614486e-06,-8.600764e-07,-0.000895,-0.000678,...,-0.007821,-0.013175,0.003263,0.025168,0.022397,0.018198,-0.000035,-0.000706,0.010331,0.018019
7,298.150940,300.186386,0.001652,0.482024,0.207725,0.317189,1.047959e-05,-2.657296e-06,-0.001475,-0.001020,...,-0.000602,-0.009430,0.003724,0.019110,0.019226,0.015106,0.004625,0.003430,0.011957,0.018585
8,298.180840,300.633049,0.001718,0.612303,0.246403,0.406686,2.709043e-05,-1.333887e-06,0.000193,-0.000927,...,-0.000828,-0.008117,-0.001176,0.005145,0.002854,-0.003733,-0.015604,-0.017871,-0.013112,-0.008933
9,298.302040,300.461037,0.001550,0.561260,0.337152,0.500936,2.518040e-05,5.151960e-07,0.002503,-0.000790,...,0.001008,0.001679,0.003366,-0.006648,-0.011467,-0.023606,-0.038762,-0.037297,-0.030485,-0.026019


In [235]:
df1.to_csv(output+'timeseries_era5_nio_as22001.csv')

In [180]:
#storms 1
mainbob22003=pd.read_csv(output+'inner/inner_nio_era5_bob22003.csv',delimiter=r",")
mainbob22003.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainbob22003=mainbob22003.drop('a', axis=1)

#pdf_bob22003=pd.read_csv(target+'2011_pmin_nio_bob22003.csv',delimiter=r",")
#pdf_bob22003.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#pdf_bob22003=pdf_bob22003.drop('a', axis=1)

#extra=pd.read_csv(output+'nio/outder_2011_nio_bob22003.csv',delimiter=r",")
#extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#extra=extra.drop('a', axis=1)


slev_bob22003=pd.DataFrame.from_dict(TCs_ts[2])
rlev_bob22003=pd.DataFrame.from_dict(TCr_ts[2])
prlev_bob22003=pd.DataFrame.from_dict(TCpr_ts[2])
prlev_bob22003['outmslp']=prlev_bob22003['outmslp']/100

#ts1_bob22003=pd.concat([mainbob22003,extra,slev_bob22003,rlev_bob22003,prlev_bob22003], axis=1, join='inner')
ts1_bob22003=pd.concat([mainbob22003,slev_bob22003,rlev_bob22003,prlev_bob22003], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[2]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[2]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[2]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[2]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_bob22003=pd.DataFrame.from_dict(alldict1)

#ts_bob22003=pd.concat([pdf_bob22003,ts1_bob22003,plev_bob22003], axis=1, join='inner')
ts_bob22003=pd.concat([ts1_bob22003,plev_bob22003], axis=1, join='inner')
ts_bob22003

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,0.640701,1.528786,296.820877,299.153351,0.000590,0.190726,0.179403,0.200639,3.696885e-06,-1.294073e-06,...,-0.220409,-0.274685,-0.288601,-0.256255,-0.212977,-0.170781,-0.112253,-0.074876,-0.044861,-0.030735
1,2.866211,1.248283,296.781807,299.553215,0.000379,0.156705,0.172438,0.143039,3.412657e-06,2.334127e-08,...,-0.229559,-0.284982,-0.288777,-0.247791,-0.192148,-0.146958,-0.090534,-0.051803,-0.021229,-0.007002
2,2.492275,-0.400551,297.020500,299.389038,0.000520,0.147498,0.108188,0.128963,1.781271e-06,1.801411e-06,...,-0.227913,-0.284392,-0.268598,-0.211320,-0.142502,-0.089433,-0.042637,-0.014458,0.007416,0.016978
3,2.475914,-0.792995,297.122368,299.428787,0.000805,0.112296,0.126034,0.132431,1.729059e-06,2.548506e-06,...,-0.216577,-0.267369,-0.245984,-0.175729,-0.105687,-0.055691,-0.035098,-0.017126,0.000982,0.008439
4,2.387924,-0.594162,297.136284,299.408958,0.000800,0.154113,0.194190,0.173135,3.889960e-06,3.241817e-06,...,-0.203466,-0.249721,-0.221730,-0.157028,-0.102540,-0.066379,-0.056544,-0.040247,-0.022861,-0.014929
5,1.993355,-0.990387,297.022453,299.407204,0.001072,0.195007,0.146537,0.176104,7.646158e-07,2.221321e-06,...,-0.186294,-0.228969,-0.209879,-0.157235,-0.113928,-0.094038,-0.081860,-0.056381,-0.030181,-0.017883
6,4.003380,-0.825661,296.782967,299.269821,0.000470,0.148003,0.111489,0.153949,1.188120e-06,1.868815e-06,...,-0.189971,-0.249374,-0.244137,-0.189105,-0.135825,-0.096929,-0.065107,-0.035046,-0.002225,0.014280
7,1.249859,-1.976860,296.848915,299.655403,0.000429,0.176648,0.180554,0.287432,1.358666e-05,7.588882e-06,...,-0.177255,-0.231727,-0.211134,-0.146545,-0.094351,-0.057173,-0.051780,-0.034464,-0.011454,-0.000209
8,-0.662830,-2.113728,296.727165,299.524277,0.000645,0.241264,0.190265,0.383332,3.671797e-06,-1.866138e-07,...,-0.170262,-0.223001,-0.212499,-0.148958,-0.095893,-0.063141,-0.069510,-0.056101,-0.035983,-0.025803
9,-2.184353,-2.809074,296.800758,299.148384,0.001455,0.377000,0.202687,0.377268,3.234702e-06,1.422944e-06,...,-0.144299,-0.178801,-0.153431,-0.101086,-0.067397,-0.052119,-0.055933,-0.038622,-0.021699,-0.014332


In [181]:
df1=ts_bob22003
#df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
#df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,vi_div_mass_flux,vi_div_moisture_flux,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,296.820877,299.153351,0.000590,0.190726,0.179403,0.200639,3.696885e-06,-1.294073e-06,0.003066,-0.000157,...,-0.220409,-0.274685,-0.288601,-0.256255,-0.212977,-0.170781,-0.112253,-0.074876,-0.044861,-0.030735
1,296.781807,299.553215,0.000379,0.156705,0.172438,0.143039,3.412657e-06,2.334127e-08,0.000866,-0.000133,...,-0.229559,-0.284982,-0.288777,-0.247791,-0.192148,-0.146958,-0.090534,-0.051803,-0.021229,-0.007002
2,297.020500,299.389038,0.000520,0.147498,0.108188,0.128963,1.781271e-06,1.801411e-06,-0.001903,-0.000201,...,-0.227913,-0.284392,-0.268598,-0.211320,-0.142502,-0.089433,-0.042637,-0.014458,0.007416,0.016978
3,297.122368,299.428787,0.000805,0.112296,0.126034,0.132431,1.729059e-06,2.548506e-06,-0.000243,-0.000268,...,-0.216577,-0.267369,-0.245984,-0.175729,-0.105687,-0.055691,-0.035098,-0.017126,0.000982,0.008439
4,297.136284,299.408958,0.000800,0.154113,0.194190,0.173135,3.889960e-06,3.241817e-06,0.002186,-0.000381,...,-0.203466,-0.249721,-0.221730,-0.157028,-0.102540,-0.066379,-0.056544,-0.040247,-0.022861,-0.014929
5,297.022453,299.407204,0.001072,0.195007,0.146537,0.176104,7.646158e-07,2.221321e-06,0.001923,-0.000263,...,-0.186294,-0.228969,-0.209879,-0.157235,-0.113928,-0.094038,-0.081860,-0.056381,-0.030181,-0.017883
6,296.782967,299.269821,0.000470,0.148003,0.111489,0.153949,1.188120e-06,1.868815e-06,-0.001554,-0.000041,...,-0.189971,-0.249374,-0.244137,-0.189105,-0.135825,-0.096929,-0.065107,-0.035046,-0.002225,0.014280
7,296.848915,299.655403,0.000429,0.176648,0.180554,0.287432,1.358666e-05,7.588882e-06,0.000280,-0.000361,...,-0.177255,-0.231727,-0.211134,-0.146545,-0.094351,-0.057173,-0.051780,-0.034464,-0.011454,-0.000209
8,296.727165,299.524277,0.000645,0.241264,0.190265,0.383332,3.671797e-06,-1.866138e-07,0.002507,-0.000552,...,-0.170262,-0.223001,-0.212499,-0.148958,-0.095893,-0.063141,-0.069510,-0.056101,-0.035983,-0.025803
9,296.800758,299.148384,0.001455,0.377000,0.202687,0.377268,3.234702e-06,1.422944e-06,0.000768,-0.000595,...,-0.144299,-0.178801,-0.153431,-0.101086,-0.067397,-0.052119,-0.055933,-0.038622,-0.021699,-0.014332


In [182]:
df1.to_csv(output+'timeseries_era5_nio_bob22003.csv')

In [154]:
#storms 1
mainagni=pd.read_csv(output+'inner/inner_nio_era5_agni.csv',delimiter=r",")
mainagni.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
mainagni=mainagni.drop('a', axis=1)

#pdf_agni=pd.read_csv(target+'2011_pmin_nio_agni.csv',delimiter=r",")
#pdf_agni.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#pdf_agni=pdf_agni.drop('a', axis=1)

#extra=pd.read_csv(output+'nio/outder_2011_nio_agni.csv',delimiter=r",")
#extra.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
#extra=extra.drop('a', axis=1)


slev_agni=pd.DataFrame.from_dict(TCs_ts[2])
rlev_agni=pd.DataFrame.from_dict(TCr_ts[2])
prlev_agni=pd.DataFrame.from_dict(TCpr_ts[2])
prlev_agni['outmslp']=prlev_agni['outmslp']/100

#ts1_agni=pd.concat([mainagni,extra,slev_agni,rlev_agni,prlev_agni], axis=1, join='inner')
ts1_agni=pd.concat([mainagni,slev_agni,rlev_agni,prlev_agni], axis=1, join='inner')

tempvortdict = {'outvort_'+plev[i]:TCp_ts[2]['outvort'][:,i] for i in range(21)}
temprhumdict = {'outrhum_'+plev[i]:TCp_ts[2]['outrhum'][:,i] for i in range(21)}
tempgpotdict = {'outgpot_'+plev[i]:TCgp_ts[2]['outgpot'][:,i] for i in range(21)}
tempvveldict = {'outvvel_'+preslv[i]:TCvv_ts[2]['outvvel'][:,i] for i in range(18)}
dict1 = {**tempvortdict,**temprhumdict}
dict3 = {**tempgpotdict,**tempvveldict}

alldict1 = {**dict1,**dict3}

plev_agni=pd.DataFrame.from_dict(alldict1)

#ts_agni=pd.concat([pdf_agni,ts1_agni,plev_agni], axis=1, join='inner')
ts_agni=pd.concat([ts1_agni,plev_agni], axis=1, join='inner')
ts_agni

,u10,v10,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,-6.209557,-0.561646,296.776299,298.953545,0.001489,0.210344,0.123156,0.075017,5.315698e-06,1.589186e-06,...,-0.017909,-0.015576,-0.024351,-0.033821,-0.023240,-0.016659,-0.015329,-0.017729,-0.017086,-0.016835
1,-6.211090,-0.348244,296.740898,299.071350,0.001398,0.252471,0.111716,0.114945,6.574555e-06,2.582325e-06,...,-0.015191,-0.009392,-0.014123,-0.017778,-0.008954,-0.012045,-0.022287,-0.026645,-0.027700,-0.027786
2,-5.952991,-0.929688,296.755478,299.370514,0.000896,0.238151,0.112947,0.111895,1.370435e-05,3.421213e-06,...,-0.026292,-0.024852,-0.034870,-0.030471,-0.017980,-0.011474,0.003097,0.006274,0.005107,0.004426
3,-4.492825,0.253735,296.932121,299.245468,0.001187,0.278084,0.105032,0.094500,1.026900e-05,3.808644e-06,...,-0.021345,-0.013755,-0.008400,0.005546,0.018190,0.028051,0.041842,0.033316,0.027007,0.025272
4,-2.311817,0.881355,296.955673,299.160782,0.001280,0.519121,0.151974,0.199504,1.101347e-05,8.253846e-08,...,-0.021849,-0.010146,0.007799,0.021609,0.022334,0.018498,0.012361,0.001393,-0.002111,-0.001146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,-4.678654,-3.036427,295.518654,297.279427,0.000062,0.003674,0.118026,0.005280,-6.862683e-08,1.584180e-06,...,0.028817,0.027777,0.041411,0.037618,0.029775,0.025786,0.027107,0.019575,0.016603,0.019697
63,-4.285057,-3.124840,295.395821,297.297943,0.000026,0.002173,0.089418,0.002694,5.218317e-08,1.939945e-06,...,0.029770,0.024170,0.043078,0.038170,0.030289,0.024312,0.034738,0.028190,0.025949,0.029530
64,-5.179806,-3.440529,295.077835,299.404930,0.000066,0.000269,0.090595,0.004278,2.299203e-08,1.200242e-06,...,0.021148,0.012058,0.028227,0.018323,0.011652,-0.000212,-0.001385,-0.005068,-0.008196,-0.007641
65,-5.125248,-2.720669,295.639069,300.428696,0.000059,0.000091,0.092147,0.008782,3.745663e-08,9.527430e-07,...,0.019470,0.010186,0.028671,0.018624,0.014646,0.002117,-0.004586,-0.010617,-0.017017,-0.019624


In [155]:
df1=ts_agni
#df1=df1.drop('ws10', axis=1)
df1=df1.drop('u10', axis=1)
df1=df1.drop('v10', axis=1)
#df1=df1.drop('out_ws10', axis=1)
df1=df1.drop('outu10', axis=1)
df1=df1.drop('outv10', axis=1)
df1=df1.drop('vi_div_ke_flux', axis=1)
df1=df1.drop('vi_div_gpot_flux', axis=1)
df1=df1.drop('vi_ke', axis=1)
df1=df1.drop('vi_div_olr_flux', axis=1)
df1=df1.drop('vi_temp', axis=1)
df1=df1.drop('inst_10m_wnd_gst', axis=1)
df1=df1.drop('surfmean_swr_flux', axis=1)
df1=df1.drop('surfmean_lwr_flux', axis=1)
df1=df1.drop('dwnwrdmean_swr_flux', axis=1)
df1=df1.drop('topmean_lwr_flux', axis=1)
df1=df1.drop('topmean_swr_flux', axis=1)
df1=df1.drop('outvi_div_ke_flux', axis=1)
df1=df1.drop('outvi_div_gpot_flux', axis=1)
df1=df1.drop('outvi_ke', axis=1)
df1=df1.drop('outvi_div_olr_flux', axis=1)
df1=df1.drop('outvi_temp', axis=1)
df1=df1.drop('outinst_10m_wnd_gst', axis=1)
df1=df1.drop('outsurfmean_swr_flux', axis=1)
df1=df1.drop('outsurfmean_lwr_flux', axis=1)
df1=df1.drop('outdwnwrdmean_swr_flux', axis=1)
df1=df1.drop('outtopmean_lwr_flux', axis=1)
df1=df1.drop('outtopmean_swr_flux', axis=1)
df1=df1.drop('outmslp', axis=1)
df1=df1.drop('mslp', axis=1)
df1

,2mdewtmp,2mtmp,conv_ppt,tot_cld_ice,tot_cldwtr,tot_cld_rain,vi_div_cld_froz_wtr,vi_div_cld_liq_wtr,vi_div_mass_flux,vi_div_moisture_flux,...,outvvel_250,outvvel_300,outvvel_400,outvvel_500,outvvel_600,outvvel_700,outvvel_850,outvvel_925,outvvel_975,outvvel_1000
0,296.776299,298.953545,0.001489,0.210344,0.123156,0.075017,5.315698e-06,1.589186e-06,0.000863,-0.000482,...,-0.017909,-0.015576,-0.024351,-0.033821,-0.023240,-0.016659,-0.015329,-0.017729,-0.017086,-0.016835
1,296.740898,299.071350,0.001398,0.252471,0.111716,0.114945,6.574555e-06,2.582325e-06,0.002410,-0.000324,...,-0.015191,-0.009392,-0.014123,-0.017778,-0.008954,-0.012045,-0.022287,-0.026645,-0.027700,-0.027786
2,296.755478,299.370514,0.000896,0.238151,0.112947,0.111895,1.370435e-05,3.421213e-06,-0.001531,-0.000392,...,-0.026292,-0.024852,-0.034870,-0.030471,-0.017980,-0.011474,0.003097,0.006274,0.005107,0.004426
3,296.932121,299.245468,0.001187,0.278084,0.105032,0.094500,1.026900e-05,3.808644e-06,-0.001852,-0.000796,...,-0.021345,-0.013755,-0.008400,0.005546,0.018190,0.028051,0.041842,0.033316,0.027007,0.025272
4,296.955673,299.160782,0.001280,0.519121,0.151974,0.199504,1.101347e-05,8.253846e-08,-0.000073,-0.000782,...,-0.021849,-0.010146,0.007799,0.021609,0.022334,0.018498,0.012361,0.001393,-0.002111,-0.001146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,295.518654,297.279427,0.000062,0.003674,0.118026,0.005280,-6.862683e-08,1.584180e-06,0.000406,0.000208,...,0.028817,0.027777,0.041411,0.037618,0.029775,0.025786,0.027107,0.019575,0.016603,0.019697
63,295.395821,297.297943,0.000026,0.002173,0.089418,0.002694,5.218317e-08,1.939945e-06,-0.000611,0.000136,...,0.029770,0.024170,0.043078,0.038170,0.030289,0.024312,0.034738,0.028190,0.025949,0.029530
64,295.077835,299.404930,0.000066,0.000269,0.090595,0.004278,2.299203e-08,1.200242e-06,0.001118,0.000102,...,0.021148,0.012058,0.028227,0.018323,0.011652,-0.000212,-0.001385,-0.005068,-0.008196,-0.007641
65,295.639069,300.428696,0.000059,0.000091,0.092147,0.008782,3.745663e-08,9.527430e-07,0.002188,0.000016,...,0.019470,0.010186,0.028671,0.018624,0.014646,0.002117,-0.004586,-0.010617,-0.017017,-0.019624


In [156]:
df1.to_csv(output+'timeseries_era5_nio_agni.csv')